## **Notebook containing scripts and outputs of the training, cross-validation and empirical data prediction for the *Drosophila* dataset**
From the manuscript Perez et al. "Species Delimitation Meets Deep Learning: Insights from a Highly Fragmented Cactus System"

In [1]:
#mount google drive to load files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import all required modules.
import numpy as np
import pandas as pd
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Input, Activation, Dense, Dropout, Flatten, concatenate
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from tensorflow.keras import regularizers
from keras import backend as K
from random import shuffle, choice

# Define parameters for the CNN run.
batch_size = 250
epochs = 250
num_classes = 2

# Define the CNN architecture.
def create_cnn(xtest, regularizer=None):
	inputShape = (xtest.shape[1], xtest.shape[2])
	inputs = Input(shape=inputShape)
	x = inputs
	x = Conv1D(250, kernel_size=2, activation='relu',input_shape=(xtest.shape[1], xtest.shape[2]))(x)
	x = Conv1D(125, kernel_size=2, activation='relu')(x)
	x = AveragePooling1D(pool_size=2)(x)
	x = Dropout(0.75)(x)
	x = Conv1D(125, kernel_size=2, activation='relu')(x)
	x = AveragePooling1D(pool_size=2)(x)
	x = Dropout(0.75)(x)
	x = Flatten()(x)
	x = Dense(125, activation='relu')(x)
	x = Dropout(0.5)(x)
	x = Dense(125, activation='relu')(x)
	x = Dropout(0.5)(x)
  # The final fully-connected layer head will have a softmax dense layer
	x = Dense(num_classes, activation="softmax")(x)

	# Construct the CNN
	model = Model(inputs, x)
	# Return the CNN
	return model

# **Train a network for each species pair with 10,000 simulations from each model**
Here we will use the full simulated dataset to train the network, by splitting the data with 75% of simulations for training and 25% for validation.

In [5]:
################################################################################################################################################
#Train a network using 10K simulations per model for the D.melanogaster-D.sechellia pair.
################################################################################################################################################
# Load Numpy arrays containing simulations.
u1 = np.load("/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation_Piloso/trainingSims/Drosophila/melano_sechellia/simModel1.npz")
u2 = np.load("/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation_Piloso/trainingSims/Drosophila/melano_sechellia/simModel2.npz")
u1 = u1["simModel1"]
u2 = u2["simModel2"]
x=np.concatenate((u1,u2),axis=0)

# Label each simulated array.
y=[0 for i in range(len(u1))]
y.extend([1 for i in range(len(u2))])
y = np.array(y)

# Print label and simulations length, these should be the same.
print (len(x), len(y))

shf = list(range(len(x)))
shuffle(shf)
y = y[shf]
x = x[shf]

# Separate train (75%) and validate (25%) sets.
xtrain, xtest = x[int(len(y)*.25):], x[:int(len(y)*.25)]
ytrain, ytest = y[int(len(y)*.25):], y[:int(len(y)*.25)]
ytest = keras.utils.to_categorical(ytest, num_classes)
ytrain = keras.utils.to_categorical(ytrain, num_classes)

# Create the CNN network, using the architecture defined above.
cnn = create_cnn(xtest)

# Compile the CNN.
cnn.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=keras.optimizers.Adam(),
	              metrics=['accuracy'])
print(cnn.summary())

# Run the CNN.
cnn.fit(xtrain, ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(xtest, ytest))

# Save the trained model
cnn.save(filepath='/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation_Piloso/Trained_10KSims_melano_sechellia.acc.mod')


20000 20000
Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 185, 5)]          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 184, 250)          2750      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 183, 125)          62625     
_________________________________________________________________
average_pooling1d_2 (Average (None, 91, 125)           0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 91, 125)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 90, 125)           31375     
_________________________________________________________________
average_pooling1d_3 (Average (None, 45, 12

In [7]:
################################################################################################################################################
#Train a network using 10K simulations per model for the D.melanogaster-D.sechellia pair.
################################################################################################################################################
# Load Numpy arrays containing simulations.
u1 = np.load("/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation_Piloso/trainingSims/Drosophila/melano_simulans/simModel1.npz")
u2 = np.load("/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation_Piloso/trainingSims/Drosophila/melano_simulans/simModel2.npz")
u1 = u1["simModel1"]
u2 = u2["simModel2"]
x=np.concatenate((u1,u2),axis=0)

# Label each simulated array.
y=[0 for i in range(len(u1))]
y.extend([1 for i in range(len(u2))])
y = np.array(y)

# Print label and simulations length, these should be the same.
print (len(x), len(y))

shf = list(range(len(x)))
shuffle(shf)
y = y[shf]
x = x[shf]

# Separate train (75%) and validate (25%) sets.
xtrain, xtest = x[int(len(y)*.25):], x[:int(len(y)*.25)]
ytrain, ytest = y[int(len(y)*.25):], y[:int(len(y)*.25)]
ytest = keras.utils.to_categorical(ytest, num_classes)
ytrain = keras.utils.to_categorical(ytrain, num_classes)

# Create the CNN network, using the architecture defined above.
cnn = create_cnn(xtest)

# Compile the CNN.
cnn.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=keras.optimizers.Adam(),
	              metrics=['accuracy'])
print(cnn.summary())

# Run the CNN.
cnn.fit(xtrain, ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(xtest, ytest))

# Save the trained model

cnn.save(filepath='/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation_Piloso/Trained_10KSims_melano_simulans.acc.mod')

20000 20000
Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 625, 6)]          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 624, 250)          3250      
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 623, 125)          62625     
_________________________________________________________________
average_pooling1d_4 (Average (None, 311, 125)          0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 311, 125)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 310, 125)          31375     
_________________________________________________________________
average_pooling1d_5 (Average (None, 155, 1

In [ ]:
################################################################################################################################################
#Evaluate the trained CNN for the D.melanogaster-D.sechellia pair using 1,000 simulations per model as test set.
################################################################################################################################################
# Load Numpy arrays containing test set simulations.
t1 = np.load("/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation/TestData/Drosophila/melano_sechellia/simModel1.npz")
t2 = np.load("/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation/TestData/Drosophila/melano_sechellia/simModel2.npz")
t1 = t1["simModel1"]
t2 = t2["simModel2"]
x=np.concatenate((t1,t2),axis=0)

# Label simulations from the test set.
y=[0 for i in range(len(t1))]
y.extend([1 for i in range(len(t2))])
y = np.array(y)

# Load the trained model.
from keras.models import load_model
from sklearn.metrics import confusion_matrix
model = load_model('/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation/Trained_10KSims_melano_sechellia.acc.mod')

# Predict and export a confusion matrix.
pred = model.predict(x)
pred_cat = [i.argmax() for i in pred]
print (confusion_matrix(y, pred_cat))
print (confusion_matrix(y, pred_cat) / float(len(y)))

[[896 104]
 [204 796]]
[[0.448 0.052]
 [0.102 0.398]]


In [ ]:
################################################################################################################################################
#Evaluate the trained CNN for the D.melanogaster-D.simulans pair using 1,000 simulations per model as test set.
################################################################################################################################################
# Load Numpy arrays containing test set simulations.
t1 = np.load("/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation/TestData/Drosophila/melano_simulans/simModel1.npz")
t2 = np.load("/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation/TestData/Drosophila/melano_simulans/simModel2.npz")
t1 = t1["simModel1"]
t2 = t2["simModel2"]
x=np.concatenate((t1,t2),axis=0)

# Label simulations from the test set.
y=[0 for i in range(len(t1))]
y.extend([1 for i in range(len(t2))])
y = np.array(y)

# Load the trained model.
from keras.models import load_model
from sklearn.metrics import confusion_matrix
model = load_model('/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation/Trained_10KSims_melano_simulans.acc.mod')

# Predict and export a confusion matrix.
pred = model.predict(x)
pred_cat = [i.argmax() for i in pred]
print (confusion_matrix(y, pred_cat))
print (confusion_matrix(y, pred_cat) / float(len(y)))

[[827 173]
 [147 853]]
[[0.4135 0.0865]
 [0.0735 0.4265]]


In [ ]:
################################################################################################################################################
#Predict the mos likely model for the empirical data, using the CNN trained with 10K simulations per model.
################################################################################################################################################
# Load the trained network.
model = load_model('/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation/Trained_10KSims_melano_sechellia.acc.mod')
# Load empirical data and transpose it.
infile=np.loadtxt("/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation/input_melano_sechellia.txt")
inp=[]
inp.append(np.array(infile).T)
x = np.array(inp)
# Predict the most likely model.
pred = model.predict(x)
print(pred)

[[0.1333712 0.8666288]]


In [ ]:
################################################################################################################################################
#Predict the mos likely model for the empirical data, using the CNN trained with 10K simulations per model.
################################################################################################################################################
# Load the trained network.
model = load_model('/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation/Trained_10KSims_melano_simulans.acc.mod')
# Load empirical data and transpose it.
infile=np.loadtxt("/content/drive/My Drive/Colab Notebooks/CNN_SpDelimitation/input_melano_simulans.txt")
inp=[]
inp.append(np.array(infile).T)
x = np.array(inp)
# Predict the most likely model.
pred = model.predict(x)
print(pred)

[[0.00256603 0.99743396]]
